In [1]:
import boto3
import os

# تنظیمات اتصال به Arvan Object Storage
access_key = "a8761df0-960e-4dd1-b5f2-ef8ef60823a9"
secret_key = "f502aad1cec94636d4381cadf302a6114df05bf825864e554b82010d6d2441ab"
region_name = "s3.ir-thr-at1.arvanstorage.ir"
bucket_name = "ehsannima"
folder_name = "xray/"  # پوشه تصاویر داخل باکت

# اتصال
s3 = boto3.client(
    "s3",
    endpoint_url=f"https://{region_name}",
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
)

# مسیر ذخیره تصاویر به صورت محلی
save_path = "./xray_images"
os.makedirs(save_path, exist_ok=True)

# لیست و دانلود تصاویر
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_name)

for obj in response.get("Contents", []):
    key = obj["Key"]
    if key.endswith((".jpg", ".png", ".jpeg")):
        filename = os.path.basename(key)
        file_path = os.path.join(save_path, filename)
        s3.download_file(bucket_name, key, file_path)
        print(f"✅ Downloaded: {filename}")


✅ Downloaded: 100(1).jpg
✅ Downloaded: 100.jpg
✅ Downloaded: 1000.jpg
✅ Downloaded: 1001.jpg
✅ Downloaded: 1002(1).jpg
✅ Downloaded: 1002.jpg
✅ Downloaded: 1003(1).jpg
✅ Downloaded: 1003.jpg
✅ Downloaded: 1004(1).jpg
✅ Downloaded: 1004.jpg
✅ Downloaded: 1005.jpg
✅ Downloaded: 1006.jpg
✅ Downloaded: 1007.jpg
✅ Downloaded: 1008.jpg
✅ Downloaded: 1009.jpg
✅ Downloaded: 101.jpg
✅ Downloaded: 1010.jpg
✅ Downloaded: 1011.jpg
✅ Downloaded: 1012.jpg
✅ Downloaded: 1013.jpg
✅ Downloaded: 1014.jpg
✅ Downloaded: 1015.jpg
✅ Downloaded: 1016.jpg
✅ Downloaded: 1017.jpg
✅ Downloaded: 1018.jpg
✅ Downloaded: 1019.jpg
✅ Downloaded: 102.jpg
✅ Downloaded: 1020.jpg
✅ Downloaded: 1021.jpg
✅ Downloaded: 1022.jpg
✅ Downloaded: 1023.jpg
✅ Downloaded: 1024.jpg
✅ Downloaded: 1025.jpg
✅ Downloaded: 1026.jpg
✅ Downloaded: 103.jpg
✅ Downloaded: 104.jpg
✅ Downloaded: 105.jpg
✅ Downloaded: 106.jpg
✅ Downloaded: 107.jpg
✅ Downloaded: 108.jpg
✅ Downloaded: 109.jpg
✅ Downloaded: 110.jpg
✅ Downloaded: 111.jpg
✅ Downloaded

In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# مسیرهای ورودی و خروجی
input_dir = './xray_images'
output_with_white = './xray_with_white'
output_without_white = './xray_without_white'

os.makedirs(output_with_white, exist_ok=True)
os.makedirs(output_without_white, exist_ok=True)

# تنظیمات
target_size = (224, 224)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

# تابع بررسی تاری تصویر (اختیاری)
def is_blurry(image, threshold=100.0):
    variance = cv2.Laplacian(image, cv2.CV_64F).var()
    return variance < threshold

# تابع حذف نوار سفید فقط از حاشیه (ایمن)
def contour_based_crop(img_gray):
    # تبدیل به باینری برعکس (ناحیه روشن = 0, پس‌زمینه = 255)
    _, thresh = cv2.threshold(img_gray, 240, 255, cv2.THRESH_BINARY_INV)

    # پیدا کردن کانتور
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return img_gray  # اگر چیزی پیدا نشد، برش نده

    # بزرگ‌ترین کانتور (بیشترین سطح)
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)

    # crop دقیق
    cropped = img_gray[y:y+h, x:x+w]
    return cropped



# پردازش تصاویر
for filename in tqdm(os.listdir(input_dir)):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(input_dir, filename)
        img = cv2.imread(img_path)

        if img is None:
            print(f"⛔ تصویر خراب: {filename}")
            continue

        # مرحله 1: Grayscale
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # مرحله 2: افزایش کنتراست با CLAHE
        img_clahe = clahe.apply(img_gray)

        # مرحله 3: حذف نویز
        img_blur = cv2.GaussianBlur(img_clahe, (3, 3), 0)

        # مرحله 4: نرمال‌سازی
        img_norm = img_blur.astype('float32') / 255.0

        # 🔵 ذخیره نسخه با نوار سفید (فقط resize)
        img_with_white = cv2.resize(img_blur, target_size)
        save_white = os.path.join(output_with_white, filename)
        cv2.imwrite(save_white, img_with_white)

        # 🟢 حذف نوار سفید از حاشیه و resize
        img_cropped = contour_based_crop(img_gray)
        img_clean = cv2.resize(img_cropped, target_size)
        save_clean = os.path.join(output_without_white, filename)
        cv2.imwrite(save_clean, img_clean)

print("\n✅ پردازش کامل شد.")
print(f"📂 با نوار سفید: {output_with_white}")
print(f"📂 بدون نوار سفید (ایمن): {output_without_white}")


100%|██████████| 134/134 [00:00<00:00, 205.36it/s]


✅ پردازش کامل شد.
📂 با نوار سفید: ./xray_with_white
📂 بدون نوار سفید (ایمن): ./xray_without_white
